In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import networkx as nx
from bokeh.plotting import figure
from bokeh.layouts import gridplot,widgetbox,column

from bokeh.io import show, output_file
from bokeh.models import Plot, GraphRenderer, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, LassoSelectTool, LabelSet, ColumnDataSource, Arrow, OpenHead, Jitter, CustomJS
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes, NodesOnly, StaticLayoutProvider
from bokeh.palettes import Spectral4, Blues8, Reds8
from bokeh.models.widgets import Div,RangeSlider


In [2]:
df_node = pd.read_csv("sheep_data/sheep_age.csv", header=0, index_col=0)
df_edge = pd.read_csv("sheep_data/sheep_edges.csv", header=0, index_col =None)

In [3]:
df_edge = pd.merge(df_edge, df_node.reset_index(), left_on='source', right_on='id').rename({'age':'source_age'}, axis='columns')
df_edge = pd.merge(df_edge, df_node.reset_index(), left_on='target', right_on='id').rename({'age':'target_age'}, axis='columns')
df_edge['older_win'] = df_edge['source_age'] > df_edge['target_age']
df_edge['older_lose'] = df_edge['source_age'] < df_edge['target_age']

In [4]:
# augmented features for each sheep node
df_node['win'] = df_edge.groupby(['source']).count()['target']
df_node['weighted_win'] = df_edge.groupby(['source']).agg({"weight": "sum"})
df_node['lose'] = df_edge.groupby(['target']).count()['source']
df_node['weighted_lose'] = df_edge.groupby(['target']).agg({"weight": "sum"})
df_node.fillna(0, inplace=True)
df_node['win'] = df_node['win'].astype(int)
df_node['lose'] = df_node['lose'].astype(int)

df_node['score'] = df_node['win'] - df_node['lose']
df_node['weighted_score'] = df_node['weighted_win'] - df_node['weighted_lose']
df_node['total'] = df_node['win'] + df_node['lose']
df_node['win_older'] = df_edge[df_edge['older_win']].groupby('source').count()['older_win']
df_node['lose_older'] = df_edge[df_edge['older_lose']].groupby('target').count()['older_lose']
df_node.fillna(0, inplace=True)
df_node.reset_index(inplace=True)
df_node['label'] = df_node.apply(lambda row: "s{}-{}/{}".format(int(row['id']), int(row['win']),int(row['lose'])), axis=1)
df_node.set_index(['id'], inplace=True)
df_node['index'] = df_node.index

In [5]:
df_node.sort_values(['age'])

,age,win,weighted_win,lose,weighted_lose,score,weighted_score,total,win_older,lose_older,label,index
id,,,,,,,,,,,,
14,1,11,31.0,11,28.0,0,3.0,22,0.0,0.0,s14-11/11,14
22,1,16,65.0,3,10.0,13,55.0,19,0.0,0.0,s22-16/3,22
18,1,10,35.0,13,25.0,-3,10.0,23,0.0,0.0,s18-10/13,18
24,1,6,25.0,11,38.0,-5,-13.0,17,0.0,0.0,s24-6/11,24
19,1,7,16.0,13,50.0,-6,-34.0,20,0.0,0.0,s19-7/13,19
6,3,0,0.0,19,62.0,-19,-62.0,19,0.0,5.0,s6-0/19,6
26,3,3,8.0,19,54.0,-16,-46.0,22,0.0,5.0,s26-3/19,26
21,3,1,10.0,21,47.0,-20,-37.0,22,0.0,4.0,s21-1/21,21
20,3,2,4.0,17,45.0,-15,-41.0,19,0.0,4.0,s20-2/17,20


In [6]:
def generate_network(df_n, df_e):
    g = nx.DiGraph()

    for i,n in df_n.iterrows():
        g.add_node(i,**n.to_dict())

    for i,e in df_e.iterrows():
        g.add_edge(e['source'], e['target'], **e.to_dict())
        #weights.append(w['weight'])
        #G.add_edge(int(e[0]),int(e[1]))
    return g

def generate_network1(df_n, df_e):
    g = nx.from_pandas_edgelist(df_e, 'source', 'target', 'weight')
    for i in sorted(G.nodes()):
        for attr in df_n.columns:
            g.node[i][attr] = df_n[attr][i]
    return g

In [7]:
df_source = df_node.sort_values(['age'])
G = generate_network(df_source, df_edge)
G1 = generate_network(df_source, df_edge)


max_score, min_score = df_source['score'].max(), df_source['score'].min()
df_source['size'] = df_source['total'] * 2
df_source['color_idx'] = df_source.apply(lambda row: 8-int(8*(row['score'])/max_score) if row['score']>0 else 7-int(8*(row['score'])/min_score), axis=1)
df_source['fill_color'] = df_source.apply(lambda row: Reds8[row['color_idx']] if row['score']>=0 else Blues8[row['color_idx']], axis=1)

In [8]:
pos = nx.circular_layout(G, scale=1, center=(0,0))
x,y = zip(*pos.values())
df_source['x'] = x
df_source['y'] = y
node_source = ColumnDataSource(df_source)
node_source_bak = ColumnDataSource(df_source)

edge_source = ColumnDataSource(data=dict(
            start=[e[0] for e in G.edges(data=True)],
            end=[e[1] for e in G.edges(data=True)],
            weight=[e[2]['weight'] for e in G.edges(data=True)],
        ))
edge_source_bak = ColumnDataSource(data=dict(
            start=[e[0] for e in G.edges(data=True)],
            end=[e[1] for e in G.edges(data=True)],
            weight=[e[2]['weight'] for e in G.edges(data=True)],
        ))

edge_source1 = ColumnDataSource(data=dict(
            start=[e[0] for e in G.edges(data=True)],
            end=[e[1] for e in G.edges(data=True)],
            weight=[e[2]['weight'] for e in G.edges(data=True)],
        ))

# HACK: important! This dummy initialization make sure the 2 edge_sources are not pointing to same object in later javascript callback
edge_source.selected['2d'] = {'indices':[]}
edge_source1.selected['2d'] = {'indices':[]}

# common renderers
hover = HoverTool(
        tooltips=[
            ("id", "@id"),
            ("age", "@age"),
            ('win', "@win"),
            ('lose', "@lose"),
            ('score', "@score"),
            ("weighted_score", "@weighted_score"),
        ]
    )
labels = LabelSet(x='x', y='y', text='label', source=node_source)

In [9]:
def plot_graph(network, title, edge_src, x_range=None, y_range=None, label=True):
    TOOLS = "pan,wheel_zoom,box_select,lasso_select"
    if not x_range:
        x_range = Range1d(-1.2,1.2)
        y_range = Range1d(-1.2,1.2)
    fig = figure(tools=TOOLS, width=600, height=600,
                x_range=x_range, y_range=y_range)
    
    fig.title.text = title

    fig.add_tools(hover, TapTool(), BoxSelectTool(), LassoSelectTool())

    graph_renderer = from_networkx(network, nx.circular_layout, scale=1, center=(0,0))
    graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=pos)

    graph_renderer.node_renderer.data_source = node_source
    graph_renderer.node_renderer.glyph = Circle(size='size', fill_color="fill_color")
    graph_renderer.node_renderer.selection_glyph = Circle(size='size', fill_color=Spectral4[2])
    graph_renderer.node_renderer.hover_glyph = Circle(size='size', fill_color=Spectral4[1])

    graph_renderer.edge_renderer.data_source = edge_src
    graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width='weight')
    graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width='weight')
    graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width='weight')

    graph_renderer.selection_policy = NodesOnly() #NodesAndLinkedEdges()
    graph_renderer.inspection_policy = NodesAndLinkedEdges() #EdgesAndLinkedNodes()

    fig.renderers.append(graph_renderer)
    fig.renderers.append(labels)
    
    return fig

In [10]:
callback = CustomJS(args=dict(edge_source=edge_source, edge_source1=edge_source1, bak=edge_source_bak), code="""
        var range = cb_obj.value;
        var bakdata = bak.data;
        
        var weights = bakdata['weight'];
        for (i = 0; i < weights.length; i++) {
            if(weights[i] < range[0] || weights[i] > range[1]) {
                edge_source.data['start'][i] = 1;
                edge_source.data['end'][i] = 1;
                edge_source.data['weight'][i] = 0;
                
                edge_source1.data['start'][i] = 1;
                edge_source1.data['end'][i] = 1;
                edge_source1.data['weight'][i] = 0;
            }else{
                edge_source.data['start'][i] = bakdata['start'][i];
                edge_source.data['end'][i] = bakdata['end'][i];
                edge_source.data['weight'][i] = bakdata['weight'][i];
                
                edge_source1.data['start'][i] = bakdata['start'][i];
                edge_source1.data['end'][i] = bakdata['end'][i];
                edge_source1.data['weight'][i] = bakdata['weight'][i];
            }
        }   
        edge_source.change.emit();
        edge_source1.change.emit();
    """)

weight_max = df_edge['weight'].max()
range_slider = RangeSlider(start=1, end=weight_max, value=(1, weight_max), step=1, title="weight", callback=callback)

In [11]:
node_source.callback = CustomJS(args=dict(edge_source=edge_source, edge_source1=edge_source1), code="""
        var node_selected = cb_obj.selected['1d']['indices'];
        var edge_starts = {};
        var edge_ends = {};
        var edges = edge_source.data;

        for(i=0;i<node_selected.length;i++) {
            ix = node_selected[i];
            node_id = cb_obj.data['index'][ix]
            for(j=0;j<edges['start'].length;j++) {
                if(edges['start'][j] == node_id) {
                    edge_starts[j] = [0];
                }
                if(edges['end'][j] == node_id) {
                    edge_ends[j] = [0];
                }
            }
        }

        edge_source.selected['2d']['indices'] = edge_starts;
        edge_source1.selected['2d']['indices'] = edge_ends;

        cb_obj.change.emit();
        edge_source.change.emit();
        edge_source1.change.emit();

    """)

In [12]:
description = '''
<h3>Node of sheep</h3>
The <b>position</b> of sheep node are ordered counter-clockwise starting from 3 o'clock <br>
The <b>size</b> of node represents the total fights number of the sheep with others <br>
The <b>Color</b> of the node shows whether the sheep has more wins(red) or more loss (blue), and the saturation indicates how many it wins or loses <br>
<h3>Edge of fight</h3>
The <b>line_width</b> indicates the dominance weight of the fight<br>
'''
div = Div(text=description, width=1000)
dummy = Div(text='')

fig = plot_graph(G, 'Sheep fight wins', edge_source)
fig1 = plot_graph(G, 'Sheep fight loses', edge_source1, fig.x_range, fig.y_range)
p = gridplot([
    [div], 
    [range_slider],
    [fig, fig1]
])

output_file("interactive_graphs.html")
show(p)

E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='02273a0d-24d9-4aaf-8ef9-df23cf1e76fa', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='09b90e4a-fddd-43d9-bf78-c7d3230dfa85', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='56ac5cad-7bf5-493c-b25c-d66e614e0b57', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='bbff0d96-b717-48d2-ba5c-71c958421b14', ...)
c:\users\wenyan\anaconda3\envs\dse241\lib\site-packages\bokeh\core\json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it wil